# Лабораторная работа: Численное интегрирование

**Раздел:** VII. Численное интегрирование (по Аристовой–Лобанову–Завьяловой)  
**Практические задачи:** VII.9, обязательная часть — задача VII.9.2

---

## Задача

Реализовать и исследовать методы численного интегрирования:

1. Квадратурные формулы Ньютона–Котеса:
   - метод прямоугольников (левые, правые, средние);
   - метод трапеций;
   - метод Симпсона (составная формула).
2. Квадратуры Гаусса–Лежандра на отрезке:
   - формулы с 2, 3 и 4 узлами.
3. Реализовать правило Рунге и экстраполяцию Ричардсона для оценки и
   уточнения результата.

В обязательной части решить одну из практических задач VII.9.  
В работе решается задача **VII.9.2**: по таблично заданной функции
вычислить значение интеграла методом Симпсона и оценить погрешность
по правилу Рунге.

Также для непрерывной функции провести численный эксперимент:
сравнить точность методов прямоугольников, трапеций, Симпсона и
квадратур Гаусса при одинаковом числе обращений к подынтегральной
функции.


## 1. Краткая теория

Рассматривается определённый интеграл

$$
I = \int_a^b f(x)\,dx.
$$

Интервал $[a,b]$ разбивается на $n$ равных частей шагом

$$
h = \frac{b-a}{n},\quad x_k = a + kh.
$$

### 1.1. Формулы Ньютона–Котеса

#### Метод прямоугольников

- Левые прямоугольники:

$$
I_h^{\text{left}} = h\sum_{k=0}^{n-1} f(x_k).
$$

- Правые прямоугольники:

$$
I_h^{\text{right}} = h\sum_{k=1}^{n} f(x_k).
$$

- Средние прямоугольники:

$$
I_h^{\text{mid}} = h\sum_{k=0}^{n-1} f\!\left(\frac{x_k + x_{k+1}}{2}\right).
$$

Порядки точности: $O(h)$ для левых/правых, $O(h^2)$ для средних.

#### Метод трапеций

$$
I_h^{\text{trap}} = \frac{h}{2}\Bigl(f(x_0) + 2\sum_{k=1}^{n-1} f(x_k) + f(x_n)\Bigr),
$$

погрешность $O(h^2)$.

#### Метод Симпсона (составная формула)

При чётном числе шагов $n=2m$:

$$
I_h^{\text{S}} = \frac{h}{3}\left[
f(x_0)
+ 4\sum_{k=1,\,\text{неч}}^{n-1} f(x_k)
+ 2\sum_{k=2,\,\text{чёт}}^{n-2} f(x_k)
+ f(x_n)
\right],
$$

погрешность $O(h^4)$.

### 1.2. Правило Рунге и экстраполяция Ричардсона

Если метод имеет порядок $p$ и

$$
I - I_h \approx C h^p,
$$

то при вычислении $I_{2h}$ и $I_h$ выполняется

$$
I - I_h \approx \frac{I_h - I_{2h}}{2^p - 1}.
$$

Соответственно:

- для метода трапеций ($p=2$): делим на $3$;
- для метода Симпсона ($p=4$): делим на $15$.

Можно также получить уточнённое значение

$$
I_{\text{R}} = I_h + \frac{I_h - I_{2h}}{2^p - 1}.
$$

### 1.3. Квадратуры Гаусса–Лежандра

На стандартном отрезке $[-1,1]$ формула имеет вид

$$
\int_{-1}^{1} f(t)\,dt \approx \sum_{k=1}^n w_k f(t_k),
$$

где узлы $t_k$ — корни полинома Лежандра $P_n(t)$, а веса $w_k$ выбраны
так, чтобы формула была точна для всех полиномов степени до $2n-1$.

Для перехода к отрезку $[a,b]$ используется замена

$$
x = \frac{a+b}{2} + \frac{b-a}{2} t,\quad
dx = \frac{b-a}{2}\,dt,
$$

и получаем

$$
\int_a^b f(x)\,dx \approx \frac{b-a}{2}\sum_{k=1}^n w_k
f\!\left(\frac{a+b}{2} + \frac{b-a}{2} t_k\right).
$$

В дальнейших ячейках реализуются все перечисленные методы и проводится
сравнение их точности.


In [9]:
import numpy as np
import matplotlib.pyplot as plt


In [10]:
def composite_left_rect(f, a, b, n):
    """Составная формула левых прямоугольников."""
    h = (b - a) / n
    x = a + h * np.arange(0, n)
    return h * np.sum(f(x))


def composite_right_rect(f, a, b, n):
    """Составная формула правых прямоугольников."""
    h = (b - a) / n
    x = a + h * np.arange(1, n + 1)
    return h * np.sum(f(x))


def composite_mid_rect(f, a, b, n):
    """Составная формула средних прямоугольников."""
    h = (b - a) / n
    x_mid = a + h * (np.arange(0, n) + 0.5)
    return h * np.sum(f(x_mid))


def composite_trap(f, a, b, n):
    """Составная формула трапеций."""
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])


def composite_simpson(f, a, b, n):
    """Составная формула Симпсона. Требуется чётное n."""
    if n % 2 != 0:
        raise ValueError("Для формулы Симпсона n должно быть чётным.")
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    S_odd = np.sum(y[1:-1:2])   # нечётные индексы
    S_even = np.sum(y[2:-1:2])  # чётные индексы
    return (h / 3) * (y[0] + 4 * S_odd + 2 * S_even + y[-1])


In [11]:
def runge_error(I_h, I_2h, p):
    """
    Оценка погрешности по правилу Рунге
    для метода порядка p.
    """
    return abs(I_h - I_2h) / (2**p - 1)


def richardson_extrapolation(I_h, I_2h, p):
    """
    Уточнение результата по Ричардсону.
    """
    return I_h + (I_h - I_2h) / (2**p - 1)


In [12]:
def gauss_legendre_nodes_weights(n):
    """
    Узлы и веса квадратур Гаусса–Лежандра на [-1, 1]
    для n = 2, 3, 4.
    """
    if n == 2:
        t = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])
        w = np.array([1.0, 1.0])
    elif n == 3:
        t = np.array([-np.sqrt(3/5), 0.0, np.sqrt(3/5)])
        w = np.array([5/9, 8/9, 5/9])
    elif n == 4:
        t1 = np.sqrt(3/7 + 2*np.sqrt(6/5)/7)
        t2 = np.sqrt(3/7 - 2*np.sqrt(6/5)/7)
        t = np.array([-t1, -t2, t2, t1])
        w1 = (18 - np.sqrt(30)) / 36
        w2 = (18 + np.sqrt(30)) / 36
        w = np.array([w1, w2, w2, w1])
    else:
        raise ValueError("Поддерживаются только n = 2, 3, 4.")
    return t, w


def gauss_legendre(f, a, b, n):
    """
    Квадратура Гаусса–Лежандра с n узлами на [a, b].
    """
    t, w = gauss_legendre_nodes_weights(n)
    # аффинное преобразование [-1,1] -> [a,b]
    x = (a + b)/2 + (b - a)/2 * t
    return (b - a)/2 * np.sum(w * f(x))


## 2. Численный эксперимент для непрерывной функции

В качестве примера возьмём интеграл

$$
I = \int_0^1 e^{-x^2}\,dx,
$$

у которого нет элементарной первообразной. В качестве «эталонного»
значения будем использовать результат формулы Симпсона на очень
мелкой сетке (например, $n = 2^{14}$ шагов).


In [13]:
def f_test(x):
    return np.exp(-x**2)

a, b = 0.0, 1.0

# "почти точное" значение через очень мелкую сетку Симпсона
n_ref = 2**14  # 16384 шагов
I_ref = composite_simpson(f_test, a, b, n_ref)
print(f"Эталонное значение I ≈ {I_ref:.10f}")

# сравним методы при умеренном числе шагов
n_list = [10, 20, 40, 80]

print("\nСравнение методов (число шагов n):")
print(f"{'n':>4} {'left':>12} {'trap':>12} {'simpson':>12} "
      f"{'Gauss-2':>12} {'Gauss-3':>12} {'Gauss-4':>12}")

for n in n_list:
    # для Симпсона n должно быть чётным — тут ок
    I_left = composite_left_rect(f_test, a, b, n)
    I_trap = composite_trap(f_test, a, b, n)
    I_simp = composite_simpson(f_test, a, b, n if n % 2 == 0 else n+1)

    # Гаусс не зависит от n шагов — только число узлов
    I_g2 = gauss_legendre(f_test, a, b, 2)
    I_g3 = gauss_legendre(f_test, a, b, 3)
    I_g4 = gauss_legendre(f_test, a, b, 4)

    print(f"{n:4d} "
          f"{abs(I_left - I_ref):12.2e} "
          f"{abs(I_trap - I_ref):12.2e} "
          f"{abs(I_simp - I_ref):12.2e} "
          f"{abs(I_g2   - I_ref):12.2e} "
          f"{abs(I_g3   - I_ref):12.2e} "
          f"{abs(I_g4   - I_ref):12.2e}")


Эталонное значение I ≈ 0.7468241328

Сравнение методов (число шагов n):
   n         left         trap      simpson      Gauss-2      Gauss-3      Gauss-4
  10     3.10e-02     6.13e-04     8.15e-07     2.29e-04     9.55e-06     3.35e-07
  20     1.56e-02     1.53e-04     5.11e-08     2.29e-04     9.55e-06     3.35e-07
  40     7.86e-03     3.83e-05     3.19e-09     2.29e-04     9.55e-06     3.35e-07
  80     3.94e-03     9.58e-06     2.00e-10     2.29e-04     9.55e-06     3.35e-07


## 3. Решение задачи VII.9.2 (табличная функция, формула Симпсона)

Дано (шаг $h = 0.25$):

- $x$: $0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2$  
- $f(x)$: $0, 0.004, 0.015, 0.034, 0.059, 0.089, 0.123, 0.3, 0.2$

Требуется:

1. Вычислить $\displaystyle I = \int_0^2 f(x)\,dx$ по составной формуле
   Симпсона.
2. Оценить погрешность по правилу Рунге, используя значения на шагах
   $h = 0.25$ и $2h = 0.5$.


In [14]:
def simpson_tabulated(x, y):
    """
    Составная формула Симпсона для табличной функции.

    Требуется:
    - узлы x отсортированы и равноотстоящие,
    - число шагов n = len(x)-1 чётное.
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    n = len(x) - 1
    if n % 2 != 0:
        raise ValueError("Число интервалов (len(x)-1) должно быть чётным.")
    h = x[1] - x[0]
    if not np.allclose(np.diff(x), h):
        raise ValueError("Узлы x должны быть равноотстоящими.")

    S_odd = np.sum(y[1:-1:2])
    S_even = np.sum(y[2:-1:2])
    return (h / 3) * (y[0] + 4 * S_odd + 2 * S_even + y[-1])


In [15]:
# Данные из задачи VII.9.2
x_tab = np.array([0, 0.25, 0.5, 0.75, 1.0,
                  1.25, 1.5, 1.75, 2.0], dtype=float)
f_tab = np.array([0, 0.004, 0.015, 0.034, 0.059,
                  0.089, 0.123, 0.3, 0.2], dtype=float)

# Формула Симпсона с шагом h = 0.25 (n = 8 интервалов)
I_h = simpson_tabulated(x_tab, f_tab)

# Формула Симпсона с удвоенным шагом 2h = 0.5:
# берём каждую вторую точку: x = [0,0.5,1.0,1.5,2.0]
x_coarse = x_tab[::2]
f_coarse = f_tab[::2]
I_2h = simpson_tabulated(x_coarse, f_coarse)

# Порядок метода Симпсона p = 4
p = 4
err_est = runge_error(I_h, I_2h, p)
I_R = richardson_extrapolation(I_h, I_2h, p)

print("Задача VII.9.2 (табличная функция)")
print(f"I_h   (h = 0.25) ≈ {I_h:.8f}")
print(f"I_2h (h = 0.5)  ≈ {I_2h:.8f}")
print(f"Оценка погрешности |I - I_h| ≈ {err_est:.8e}")
print(f"Уточнённое по Ричардсону значение I_R ≈ {I_R:.8f}")


Задача VII.9.2 (табличная функция)
I_h   (h = 0.25) ≈ 0.19183333
I_2h (h = 0.5)  ≈ 0.14500000
Оценка погрешности |I - I_h| ≈ 3.12222222e-03
Уточнённое по Ричардсону значение I_R ≈ 0.19495556


## 4. Дополнительный эксперимент: правило Рунге

Для выбранной непрерывной функции $f(x) = e^{-x^2}$ на отрезке $[0,1]$
используем правило Рунге:

- для метода трапеций (порядок $p=2$),
- для формулы Симпсона (порядок $p=4$),

чтобы оценить погрешность и уточнить результат.


In [16]:
n_h = 40
n_2h = n_h // 2  # удвоенный шаг

# трапеции
I_trap_h = composite_trap(f_test, a, b, n_h)
I_trap_2h = composite_trap(f_test, a, b, n_2h)
err_trap = runge_error(I_trap_h, I_trap_2h, p=2)
I_trap_R = richardson_extrapolation(I_trap_h, I_trap_2h, p=2)

# Симпсон (n_h и n_2h должны быть чётными)
if n_h % 2 != 0:
    n_h += 1
if n_2h % 2 != 0:
    n_2h += 1

I_simp_h = composite_simpson(f_test, a, b, n_h)
I_simp_2h = composite_simpson(f_test, a, b, n_2h)
err_simp = runge_error(I_simp_h, I_simp_2h, p=4)
I_simp_R = richardson_extrapolation(I_simp_h, I_simp_2h, p=4)

print("Правило Рунге для f(x)=exp(-x^2) на [0,1]:")
print(f"Трапеции: I_h = {I_trap_h:.10f}, I_2h = {I_trap_2h:.10f}")
print(f"  Оценка погрешности ≈ {err_trap:.2e}")
print(f"  Уточнение I_R ≈ {I_trap_R:.10f}, |I_R - I_ref| ≈ {abs(I_trap_R - I_ref):.2e}")

print(f"\nСимпсон:  I_h = {I_simp_h:.10f}, I_2h = {I_simp_2h:.10f}")
print(f"  Оценка погрешности ≈ {err_simp:.2e}")
print(f"  Уточнение I_R ≈ {I_simp_R:.10f}, |I_R - I_ref| ≈ {abs(I_simp_R - I_ref):.2e}")


Правило Рунге для f(x)=exp(-x^2) на [0,1]:
Трапеции: I_h = 0.7467858112, I_2h = 0.7466708369
  Оценка погрешности ≈ 3.83e-05
  Уточнение I_R ≈ 0.7468241360, |I_R - I_ref| ≈ 3.19e-09

Симпсон:  I_h = 0.7468241360, I_2h = 0.7468241839
  Оценка погрешности ≈ 3.19e-09
  Уточнение I_R ≈ 0.7468241328, |I_R - I_ref| ≈ 1.55e-12


## 5. Выводы

1. Составные формулы Ньютона–Котеса (прямоугольники, трапеции,
   Симпсон) позволяют вычислять интегралы по табличным значениям
   функции. Метод Симпсона имеет порядок точности $O(h^4)$ и даёт
   значительно меньшую погрешность по сравнению с методом трапеций
   при том же шаге.

2. В задаче VII.9.2 формула Симпсона по табличным данным даёт значение
   интеграла $I_h$, а использование шага $2h$ и правила Рунге позволяет
   оценить погрешность и получить уточнённое значение $I_R$.

3. Квадратуры Гаусса–Лежандра при малом числе узлов (2–4) обеспечивают
   очень высокую точность для гладких функций, поскольку имеют
   алгебраический порядок точности $2n-1$ (для $n$ узлов).

4. Правило Рунге и экстраполяция Ричардсона являются удобным средством
   практической оценки погрешности и повышения точности результата без
   существенного увеличения числа вычислений.
